# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2025-06-01	Ancient Anorthosites on Mars: Widespread in the Lower Crust	Research Square (Preprint)	Discovery of widespread ancient anorthosite in Mars' lower crust, providing new insights into early planetary differentiation and crustal evolution.	Phillips, M., Viviano, C., Rogers, A.D., Larson, L., Tornabene, L., Trowbridge, A., Moersch, J., McSween, H. (2025). "Ancient Anorthosites on Mars: Widespread in the Lower Crust." <i>Research Square</i>.	ancient-anorthosites-mars-2025	http://dx.doi.org/10.21203/rs.3.rs-6823461/v1	
2024-11-01	HyPyRameter: A Python Toolbox to Calculate Spectral Parameters from Hyperspectral Reflectance Data	The Planetary Science Journal	Comprehensive Python toolkit for calculating spectral parameters from hyperspectral data, designed for planetary science applications.	Phillips, M.S., Udovicic, C.T., Moersch, J.E., Basu, U., Hamilton, C.W. (2024). "HyPyRameter: A Python Toolbox to Calculate Spec

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2025-06-01,Ancient Anorthosites on Mars: Widespread in th...,Research Square (Preprint),Discovery of widespread ancient anorthosite in...,"Phillips, M., Viviano, C., Rogers, A.D., Larso...",ancient-anorthosites-mars-2025,http://dx.doi.org/10.21203/rs.3.rs-6823461/v1,NaN
1,2024-11-01,HyPyRameter: A Python Toolbox to Calculate Spe...,The Planetary Science Journal,Comprehensive Python toolkit for calculating s...,"Phillips, M.S., Udovicic, C.T., Moersch, J.E.,...",hypyrameter-python-toolbox-2024,NaN,NaN
2,2024-08-01,Comparing Rover and Helicopter Planetary Missi...,The Planetary Science Journal,Field study comparing rover and helicopter mis...,"Gwizd, S., Stack, K.M., Francis, R., Calef, F....",rover-helicopter-mars-analog-iceland-2024,http://dx.doi.org/10.3847/PSJ/ad55f4,NaN
3,2024-03-01,New Views into Pre-Noachian Mars,LPI Contributions,Investigation of ancient Martian crustal compo...,"Viviano, C.E., Phillips, M.S. (2024). ""New Vie...",new-views-pre-noachian-mars-2024,NaN,NaN
4,2024-03-01,Spectral Cube Analysis Tool: A Python Program ...,LPI Contributions,Development of Python software for processing ...,"Phillips, M.S. (2024). ""Spectral Cube Analysis...",spectral-cube-analysis-tool-2024,NaN,NaN
5,2024-03-01,The Space Imagery Center at the University of ...,LPI Contributions,Overview of the Space Imagery Center facilitie...,"Phillips, M.S., Byrne, S. (2024). ""The Space I...",space-imagery-center-arizona-2024,NaN,NaN
6,2024-03-01,Interpreting Plagioclase Detections in Igneous...,LPI Contributions,Spectroscopic analysis comparing terrestrial a...,"Phillips, M.S., Vannier, H., Horgan, B., Phill...",plagioclase-detections-igneous-spectra-2024,NaN,NaN
7,2024-02-01,Bottlenecks and Next Steps in the Integration ...,The Astrobiology Science Conference (AbSciCon)...,Analysis of current limitations and future dir...,"Warren-Rhodes, K., Phillips, M., Gong, J., Hof...",ai-ml-astrobiology-bottlenecks-2024,NaN,NaN
8,2023-09-01,Gypsum-lined degassing holes in tumuli,Earth Surface Processes and Landforms,Study of gypsum formations in volcanic tumuli ...,"Hofmann, M.H., Hinman, N.W., Phillips, M., McI...",gypsum-degassing-holes-tumuli-2023,https://doi.org/10.1002/esp.5692,NaN
9,2023-07-01,A first look at CRISM hyperspectral mapping mo...,Icarus,First comprehensive analysis of CRISM hyperspe...,"Phillips, M.S., Murchie, S.L., Seelos, F.P., H...",crism-hyperspectral-mawrth-vallis-2023,https://doi.org/10.1016/j.icarus.2023.115712,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [4]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [5]:
!ls ../_publications/

2018-03-01-astrobiology-fdl-partnership-2018.md
2018-03-01-feldspathic-rock-hellas-mantle-2018.md
2018-09-01-hydrothermal-alteration-impact-basins-2018.md
2019-03-01-hydrothermal-alteration-impact-basins-2019.md
2019-03-01-plagioclase-ancient-crust-hellas-2019.md
2019-07-01-microbial-communities-salar-pajonales-2019.md
2020-03-01-temporal-analysis-cerro-pasco-2020.md
2021-01-01-biosignatures-knowledge-gaps-mars-2021.md
2021-12-01-regional-composition-isidis-hellas-2021.md
2022-04-01-automated-detection-mars-analog-2022.md
2022-04-01-gypsum-habitability-zones-2022.md
2022-05-01-crism-radiometric-reconciliation-2022.md
2022-06-01-astrobiological-mars-exploration-2022.md
2022-06-01-mars-analog-deep-learning-2022.md
2022-06-01-mars-uplifted-massifs-ancient-crust-2022.md
2023-03-01-crism-processing-pipeline-2023.md
2023-07-01-crism-hyperspectral-mawrth-vallis-2023.md
2023-09-01-gypsum-degassing-holes-tumuli-2023.md
2023-10-01-ai-search-for-life-2023.md
2023-10-01-basin-related-alteration-ea

In [6]:
!cat ../_publications/2018-03-01-astrobiology-fdl-partnership-2018.md

---
title: "Advancing Astrobiology Through Public/Private Partnership: The FDL Model"
collection: publications
permalink: /publication/2018-03-01-astrobiology-fdl-partnership-2018
excerpt: 'Discussion of the Frontier Development Lab model for advancing astrobiology research through public-private partnerships and machine learning.'
date: 2018-03-01
venue: '49th Annual Lunar and Planetary Science Conference'
citation: 'Cabrol, N.A., Diamond, W.H., Altaf, N., Bishop, J., Cady, S.L., Fenton, L., Hinman, N., Jain, S., Lowndes, A., Mackintosh, G., Phillips, M., et al. (2018). &quot;Advancing Astrobiology Through Public/Private Partnership: The FDL Model.&quot; <i>49th LPSC</i>. 1275.'
---
Discussion of the Frontier Development Lab model for advancing astrobiology research through public-private partnerships and machine learning.

Recommended citation: Cabrol, N.A., Diamond, W.H., Altaf, N., Bishop, J., Cady, S.L., Fenton, L., Hinman, N., Jain, S., Lowndes, A., Mackintosh, G., Phillips, M., 